In [1]:
from gurobipy import * # requires gurobi package installed
import sys
import numpy as np
import math
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.table import Table
import time
import pickle

In [2]:
import sys 
sys.path.append('..') # to add the parent directory to be able to import nomadic_trucker_new

from nomadic_trucker_new import nomadictrucker

In [3]:
i_gridDim = 3
i_trailerTypes = 3
i_days = 3
model = nomadictrucker(i_gridDim, i_trailerTypes, i_days)
model.keys()

dict_keys(['size', 'num_days', 'num_trailers', 'get_first_state', 'get_distance', 'get_actual_next_grid_state', 'states', 'all_actions', 'get_rewards', 'get_numeric_state_nomadic_trucker', 'get_state_exp', 'num_possible_actions'])

In [4]:
# len(model['states'])
# model['get_numeric_state_nomadic_trucker'](1, 2, 2, [0, 0, 0, 0, 0, 0, 0, 0, 0])

In [5]:
# model['get_numeric_state_nomadic_trucker'](1, 2, 2, [1, 1, 1, 1, 1, 1, 1, 1, 1])

In [6]:
# model['get_numeric_state_nomadic_trucker'](1, 2, 2, [0]*9)

In [7]:
[1]*9

[1, 1, 1, 1, 1, 1, 1, 1, 1]

In [8]:
19455-18944

511

In [9]:
# rewards
r=[]
for i in range((len(model['all_actions']))):
    for j in range((len(model['states']))):
        r.append(model['get_rewards'](j,i))

rewards = np.reshape(r, (len(model['all_actions']),len(model['states'])))
print(len(rewards))
print(len(rewards[0]))

9
41472


In [10]:
# actions
a = (model['all_actions'])
actions = [a]*(len(model['states']))

print('num_states', len(model['states']))
print(model['all_actions'])
print('num_actions', len(actions))
print('actions[0]', actions[0])

num_states 41472
[0, 1, 2, 3, 4, 5, 6, 7, 8]
num_actions 41472
actions[0] [0, 1, 2, 3, 4, 5, 6, 7, 8]


In [11]:
(0 + 1)%model['num_days']

1

In [12]:
41472/9

4608.0

In [13]:
# model['size']
# 1/(2**(model['size']))

# model['get_state_exp'](4100)

In [14]:
def get_transition_probs(action, from_state, to_state):
    from_state_day = model['get_state_exp'](from_state)[1]
    from_state_trailer = model['get_state_exp'](from_state)[2]
    to_state_location = model['get_state_exp'](to_state)[0]
    to_state_day = model['get_state_exp'](to_state)[1]
    to_state_trailer = model['get_state_exp'](to_state)[2]
    if (to_state_location == action and to_state_day == (from_state_day%(model['num_days']) + 1) and to_state_trailer == (from_state_trailer%(model['num_trailers']) + 1)):
        return 1/(2**(model['size']))
        # return 1
    else:
        return 0

In [15]:
def LP_method_contruct_model(giNumStates, gvActions, gvRews, gdDiscountFactor=0.9):
    
    flat_list = [item for sublist in gvActions for item in sublist] # flatten the list of lists
    t_iNumActions = len(set(flat_list))
    # model declaration
    peModel = Model("lp_model")

    # model variables are defined here
    v = peModel.addVars(giNumStates, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS)

    # set the objective function for the LP model
    t_vDelta = [1.0/float(giNumStates)]*giNumStates
    
    print("start: LP model construction")
    expr = 0
    for s in range(giNumStates):
        expr += t_vDelta[s]*v[s]

    peModel.setObjective(expr,GRB.MINIMIZE)

    t_dFixedTrProb = 1.0/(2**(model['size']))

    # set all the constraints
    for s in range(giNumStates):

        # print progress at every 1000 states
        if (s % 1000 == 0):
            print("state ", s)

        # mc modifications
        from_state_location = model['get_state_exp'](s)[0]
        from_state_day = model['get_state_exp'](s)[1]
        from_state_trailer = model['get_state_exp'](s)[2]


        for a in gvActions[s]:

            # mc modifications
            to_state_location = a
            to_state_day = 1+ ((from_state_day + 1)%model['num_days']) # because of the cyclic nature
            to_state_trailer = 1+((from_state_trailer + 1))%model['num_trailers'] # because of the cyclic nature

            # print("to_state_location", "to_state_day", "to_state_trailer", to_state_location, to_state_day, to_state_trailer)
            to_state_start_index = model['get_numeric_state_nomadic_trucker'](to_state_location, to_state_day, to_state_trailer, [0]*model['size'])
            # print("to_state_start_index", to_state_start_index)

            expr = v[s]
            for ss in range(to_state_start_index, to_state_start_index + 2**(model['size'])):
                expr -= gdDiscountFactor*t_dFixedTrProb*v[ss]

            # equations are stored in an expression
            # expr = v[s]
            # for ss in range(giNumStates):
            #     expr -= gdDiscountFactor*get_transition_probs(a, s, ss)*v[ss]
            #     # expr -= gdDiscountFactor*0.001 # this is fast, so table look up is needed

            # adding constraints to the model
            peModel.addConstr(expr >= gvRews[a][s])

    print("end: LP model construction")

    return peModel, v

In [16]:
def LP_method_solve_model(gModel, gvars):
    # write the model to a file
    # peModel.write('lpOut_opt_instance.lp')

    # gurobi parameter setting
    # gModel.setParam(GRB.Param.TimeLimit, 100.0)
    gModel.setParam(GRB.Param.Threads, 7) #
    gModel.setParam(GRB.Param.Method, 2) # Options are: -1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex.
    # gModel.setParam(GRB.Param.Presolve, 0) #
    # gModel.setParam(GRB.Param.MIPFocus, 1)
    # gModel.setParam(GRB.Param.ImproveStartTime, 1800)
    # gModel.setParam(GRB.Param.SolutionLimit, 1)


    # solve the model
    print("begin: solve LP model")
    gModel.optimize()

    # print the objective value
    print("objVal" + '\t' + str(gModel.objVal))

    # print the variable values
    tmp_vsol = gModel.getAttr('x', gvars)
    # tmp_vsol = gModel.getAttr('x')
    print("solution" + '\t' + str(tmp_vsol))

    return tmp_vsol

In [17]:
# construct the model
start = time.time()
lp_model, v_vars = LP_method_contruct_model(len(model['states']), actions, rewards)
end = time.time()

print("model construction time", np.round(end - start,3))

Academic license - for non-commercial use only
start: LP model construction
state  0
state  1000
state  2000
state  3000
state  4000
state  5000
state  6000
state  7000
state  8000
state  9000
state  10000
state  11000
state  12000
state  13000
state  14000
state  15000
state  16000
state  17000
state  18000
state  19000
state  20000
state  21000
state  22000
state  23000
state  24000
state  25000
state  26000
state  27000
state  28000
state  29000
state  30000
state  31000
state  32000
state  33000
state  34000
state  35000
state  36000
state  37000
state  38000
state  39000
state  40000
state  41000
end: LP model construction
model construction time 181.355


In [18]:
start = time.time()
v_vals = LP_method_solve_model(lp_model, v_vars)
end = time.time()

print("model solution time", np.round(end - start))

, 40730: 13650.169932545667, 40731: 13650.169932545667, 40732: 13650.169932545667, 40733: 13650.169932545667, 40734: 13650.169932545667, 40735: 13650.169932545667, 40736: 13650.169932545667, 40737: 13650.169932545667, 40738: 13650.169932545667, 40739: 13650.169932545667, 40740: 13650.169932545667, 40741: 13650.169932545667, 40742: 13650.169932545667, 40743: 13650.169932545667, 40744: 13650.169932545667, 40745: 13650.169932545667, 40746: 13650.169932545667, 40747: 13650.169932545667, 40748: 13650.169932545667, 40749: 13650.169932545667, 40750: 13650.169932545667, 40751: 13650.169932545667, 40752: 13650.169932545667, 40753: 13650.169932545667, 40754: 13650.169932545667, 40755: 13650.169932545667, 40756: 13650.169932545667, 40757: 13650.169932545667, 40758: 13650.169932545667, 40759: 13650.169932545667, 40760: 13650.169932545667, 40761: 13650.169932545667, 40762: 13650.169932545667, 40763: 13650.169932545667, 40764: 13650.169932545667, 40765: 13650.169932545667, 40766: 13650.169932545667,

In [19]:
gamma = 0.9

In [20]:
t_dFixedTrProb = 1.0/(2**(model['size']))

pi_s = [None]*len(model['states'])
for state_index in model['states']:
    max_value = -float('inf')
    best_action = None
    new_V_s = 0

    # mc modifications
    from_state_location = model['get_state_exp'](state_index)[0]
    from_state_day = model['get_state_exp'](state_index)[1]
    from_state_trailer = model['get_state_exp'](state_index)[2]

    for action_index, action in enumerate(model['all_actions']):
        new_V_s = model['get_rewards'](state_index, action)

        # mc modifications
        to_state_location = action_index
        to_state_day = 1+ ((from_state_day + 1)%model['num_days']) # because of the cyclic nature
        to_state_trailer = 1+((from_state_trailer + 1))%model['num_trailers'] # because of the cyclic nature

        # print("to_state_location", "to_state_day", "to_state_trailer", to_state_location, to_state_day, to_state_trailer)
        to_state_start_index = model['get_numeric_state_nomadic_trucker'](to_state_location, to_state_day, to_state_trailer, [0]*model['size'])
        # print("to_state_start_index", to_state_start_index)

        for next_state_index in range(to_state_start_index, to_state_start_index + 2**(model['size'])):
            new_V_s -= gamma*t_dFixedTrProb*v_vals[next_state_index]
        
        # # mc: this needs to be modified similar to LP_method_contruct_model() function
        # for next_state_index, next_state in enumerate(model['states']):
        #     new_V_s += gamma * get_transition_probs(action, state_index, next_state_index)*v_vals[next_state_index]

        if new_V_s > max_value:
            best_action = action
            max_value = new_V_s

    pi_s[state_index] = best_action

In [21]:
#save as pickle file
pickle.dump( dict(v_vals), open( "V_s_"+ str(len(model['states'])) +".pickle", "wb" ) )
pickle.dump( pi_s, open( "pi_"+ str(len(model['states'])) +".pickle", "wb" ) )

In [22]:
#Open files
# V_s = pickle.load(open("V_s_1344.pickle","rb"))
# pi_s = pickle.load(open("LP_Results/pi_64.pickle","rb"))